In [6]:
from IPython.display import Audio
from datasets import load_dataset

In [1]:
ds = load_dataset("Kyudan/MathBridge")
ds_train = ds["train"]

In [18]:
# text = ds_train[442320]["spoken_English"]
text = "h prime of x is equal to d over d x of sine x plus d over d x of square root of 3 times cosine x"
text

'h prime of x is equal to d over d x of sine x plus d over d x of square root of 3 times cosine x'

In [19]:
from TTS.api import TTS

# Initialize TTS with the model name
model_name1 = "tts_models/en/ljspeech/speedy-speech"
model_name2 = "tts_models/en/ljspeech/fast_pitch"

tts1 = TTS(model_name1)
tts2 = TTS(model_name2)

# Test conversion with a sample sentence
tts1.tts_to_file(text=text, file_path="./speech/tts1.wav")
tts2.tts_to_file(text=text, file_path="./speech/tts2.wav")

 > tts_models/en/ljspeech/speedy-speech is already downloaded.
 > vocoder_models/en/ljspeech/hifigan_v2 is already downloaded.
 > Using model: speedy_speech
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Vocoder Model: hifigan
 > Setting up Audio Processor...
 | > sample_rate:22050
 | >

'./speech/tts2.wav'

In [20]:
Audio("./speech/tts1.wav")

In [21]:
Audio("./speech/tts2.wav")